Goal for 6/25
    - add logic for checking year
    - add logic for comparing education
    - add logic to parse work summary
    
Goal for 8/7
    - use people parser for given names
    - try to run people with university of virginia as school
    
Goal for 8/8
    - make it still output first four columns when cannot find or parse
    - kick out people it cannot parse into it's own file
    - add company as a default search paramter
    - make separate list of people 

In [11]:
from selenium import webdriver
import csv
import time
import random
import pandas as pd
import probablepeople as pp
import numpy as np

In [3]:
df = pd.read_excel('alumni_sample902-1001.xlsx')

In [4]:
df.head()

,ENTITY_ID_NUMBER,PREF_MAIL_NAME,YEAR1,BUSINESS_TITLE,EMPLOYER
0,2988279,Mr. Jared D. Hahn,2012,NaN,NaN
1,2988784,Mr. Weston M. Gardner,2012,NaN,NaN
2,2854156,Mr. Seunghyun No,2006,NaN,NaN
3,2934077,Ms. Kiran A. Jinsi,2010,NaN,NaN
4,267559,Mr. Joseph A. Readyhough,1998,NaN,NaN


In [20]:
df = df.fillna('')

In [21]:
df.head()

,ENTITY_ID_NUMBER,PREF_MAIL_NAME,YEAR1,BUSINESS_TITLE,EMPLOYER
0,2988279,Mr. Jared D. Hahn,2012,,
1,2988784,Mr. Weston M. Gardner,2012,,
2,2854156,Mr. Seunghyun No,2006,,
3,2934077,Ms. Kiran A. Jinsi,2010,,
4,267559,Mr. Joseph A. Readyhough,1998,,


In [18]:
p = pp.tag('Ms. Shondale Terree Bostick')

In [44]:
p

(OrderedDict([('CorporationName', 'Ms. Shondale Terree Bostick')]),
 'Corporation')

In [5]:
url = 'https://www.linkedin.com/'
username = 'mariamariasmithsmithsmith@gmail.com'
password = 'scrapenrape'

In [6]:
driver = webdriver.Chrome()

In [7]:
driver.get(url)

In [8]:
driver.find_element_by_id('login-email').send_keys(username)
time.sleep(random.gauss(1, .1))
driver.find_element_by_id('login-password').send_keys(password)
time.sleep(random.gauss(1, .1))
driver.find_element_by_name('submit').click()

In [9]:
school = 'University of Virginia'
REPLACE_CHARS = ['1st','2nd','3rd',' CPA','Premium Badge','GROUP']

In [22]:
unparsed_info = []
unfound_info = []
scraped_info = []
for idx, row in df.iterrows():
    print(row.EMPLOYER)
    entity_id = row.ENTITY_ID_NUMBER
    orig_name = row.PREF_MAIL_NAME
    try:    
        parsed_name = pp.tag(orig_name)
        first_name = parsed_name[0]['GivenName']
        last_name = parsed_name[0]['Surname']
    except Exception:
        info = (entity_id, orig_name, grad_year, business_title, employer)
        unparsed_info.append(list(info))
        continue
        
    grad_year = str(row.YEAR1)
    business_title = row.BUSINESS_TITLE
    employer = row.EMPLOYER
    
    print("Processing {}: {} {}".format(idx, first_name, last_name))
    
    driver.find_element_by_id('advanced-search').click()
    time.sleep(random.gauss(1, .1))
    
    driver.find_element_by_id('advs-firstName').clear()
    driver.find_element_by_id('advs-lastName').clear()
    driver.find_element_by_id('advs-school').clear()
    driver.find_element_by_id('advs-company').clear()
    
    driver.find_element_by_id('advs-firstName').send_keys(first_name)
    driver.find_element_by_id('advs-lastName').send_keys(last_name)
    driver.find_element_by_id('advs-company').send_keys(employer)
    driver.find_element_by_id('advs-school').send_keys(school)
    driver.find_element_by_class_name('submit-advs').click()
    time.sleep(random.gauss(4.2, .5))
    
    num_results = driver.find_element_by_class_name('search-info').text.split('\n')[0]
    num_results = num_results.strip(" results")
    #num_results = int(num_results)
    if num_results == '0':
        info = (entity_id, orig_name, grad_year, business_title, employer, first_name, last_name)
        unfound_info.append(list(info))
        continue
        
    results_summary = driver.find_element_by_id('results-container').text.split('\n')
    #print('results summary: {}'.format(results_summary))
    scraped_name = results_summary[0]
    for chars in REPLACE_CHARS:
        scraped_name = scraped_name.replace(chars, '')
    #print('scraped name: {}'.format(scraped_name))
    summary_position = results_summary[1]
    
    full_name = first_name + " " + last_name
    name_match = "True" if scraped_name == full_name else "False"
    
    driver.find_element_by_css_selector('a.title.main-headline').click()
    time.sleep(random.gauss(1.1, .2))
    
    try:
        education_text = driver.find_element_by_id('background-education').text.replace('Education\n', '')
        school_match = "True" if school in education_text else "False"
        year_match = "True" if grad_year in education_text else "False"
        parsed_education_text = education_text.split('\n')
    except Exception:
        education_text = parsed_education_text = ''
        school_match = year_match = "False"
    
    try:
        work_experience = driver.find_element_by_id('background-experience-container').text.replace('Experience\n', '')
        parsed_work_experience = work_experience.split('\n')
        current_position = parsed_work_experience[0]
        current_company = parsed_work_experience[1]
        current_location = parsed_work_experience[2].rsplit(')')[1]
        employer_match = "True" if current_company == employer else "False"
    except Exception:
        parsed_work_experience = current_position = current_company = current_location = ''
        employer_match = "False"
        
    #if name_match == 'False' and school_match == 'False' and year_match == 'False':
    #    try_mcintyre(driver=driver, first_name, last_name, school=school2)
    
    driver.find_element_by_id('advanced-search').click()
    time.sleep(random.gauss(1.1, .1))
    
    info = (entity_id, orig_name, grad_year, business_title, employer, first_name, last_name, num_results, scraped_name, summary_position,
            current_position, current_company, current_location,
            parsed_education_text, parsed_work_experience, name_match, school_match, year_match, employer_match)
    scraped_info.append(list(info))
    
    


Processing 0: Jared Hahn

Processing 1: Weston Gardner

Processing 2: Seunghyun No


Processing 4: Joseph Readyhough

Processing 5: Marcia McHenry

Processing 6: Matthew Adinolfi

Processing 7: Kathryn Young

Processing 8: Joseph Hooker

Processing 9: Edwin Wong

Processing 10: Karl Brimmer

Processing 11: Andrew Martin

Processing 12: Margaret Cleary

Processing 13: Mary Alnutt

Processing 14: Alexander Wu

Processing 15: Victoria Wolf

Processing 16: Russell Lewis



Processing 19: Tiffany Mitchell


Processing 21: Nancy Hopkins

Processing 22: Suzanne Boggs

Processing 23: Key Michel

Processing 24: Megan Collins


Processing 26: Daniela Swaby

Processing 27: Beth Goldstein

Processing 28: Matthew Fania

Processing 29: Wensi Wu


Processing 31: Keith Haussermann


Processing 33: Christopher Arnwine

Processing 34: Kyle Rudman

Processing 35: Mark Norman

Processing 36: Anne Buettner

Processing 37: Andrea Demasi

Processing 38: Minwoo Nam

Processing 39: Kathryn Wicker

Processing 

In [24]:
with open('results/902-1001/results902-1001.csv', 'w') as output:
    scraper_writer = csv.writer(output)
    scraper_writer.writerow(['ENTITY_ID_NUMBER','PREF_MAIL_NAME','YEAR1','BUSINESS_TITLE','EMPLOYER',
                    'first_name','last_name','No. Search Results','Search Results Name',
                    'Search Results Work Summary', 'Current Position','Current Company','Current Location',
                    'Education Experience','Work Experience','Name Match','School Match','Year Match',
                    'Employer Match'])
    for row in scraped_info:
        scraper_writer.writerow(row)

In [25]:
with open('results/902-1001/unfound.csv', 'w') as no_output:
    scraper_writer = csv.writer(no_output)
    scraper_writer.writerow(['ENTITY_ID_NUMBER','PREF_MAIL_NAME','YEAR1','BUSINESS_TITLE','EMPLOYER',
                    'first_name','last_name'])
    for row in unfound_info:
        scraper_writer.writerow(row)

In [26]:
with open('results/902-1001/unparsed.csv', 'w') as no_parse:
    scraper_writer = csv.writer(no_parse)
    scraper_writer.writerow(['ENTITY_ID_NUMBER','PREF_MAIL_NAME','YEAR1','BUSINESS_TITLE','EMPLOYER'])
    for row in unparsed_info:
        scraper_writer.writerow(row)

In [12]:
"""
dfs = []
for info in scraped_info:
    print(info)
    df = pd.DataFrame(info, columns=['ENTITY_ID_NUMBER','PREF_MAIL_NAME','YEAR1','BUSINESS_TITLE','EMPLOYER',
                    'first_name','last_name','No. Search Results','Search Results Name',
                    'Search Results Work Summary', 'Current Position','Current Company','Current Location',
                    'Education Experience','Work Experience','Name Match','School Match','Year Match'])
"""

In [27]:
driver.close()

In [27]:
unfound_df = pd.read_csv('results/358-476/unfound.csv')

In [28]:
unfound_df.head()

,ENTITY_ID_NUMBER,PREF_MAIL_NAME,YEAR1,BUSINESS_TITLE,EMPLOYER,first_name,last_name
0,194358,Mr. Jonathan M. Housley,1991,President,John Housley Real Estate,Jonathan,Housley
1,189127,Mr. Charles Anthony Verbeck,1990,Director of Supply Chain,Johnson & Johnson,Charles,Verbeck
2,94181,Miss Elaine M. Schweitzer,1980,"VP, Financial Plng. & Performance","Kaiser Foundation Health Plan, Inc.",Miss Elaine,Schweitzer
3,2767945,Mrs. Kimberly A. Crowell,2003,Principal,"Kalo Holdings, LLC",Kimberly,Crowell
4,2730804,Mr. Christopher D. Lawrence,1999,Manager/Auditor,Kearney-National Incorporated,Christopher,Lawrence


In [29]:
driver = webdriver.Chrome()

In [30]:
driver.get(url)

In [31]:
driver.find_element_by_id('login-email').send_keys(username)
driver.find_element_by_id('login-password').send_keys(password)
time.sleep(random.gauss(1, .1))
driver.find_element_by_name('submit').click()

In [33]:
unfound_info = []
scraped_info = []
unyearmatch_info = []
for idx, row in unfound_df.iterrows():
    entity_id = row.ENTITY_ID_NUMBER
    orig_name = row.PREF_MAIL_NAME
    first_name = row.first_name
    last_name = row.last_name    
    grad_year = str(row.YEAR1)
    business_title = row.BUSINESS_TITLE
    employer = row.EMPLOYER
    
    print("Processing {}: {} {}".format(idx, first_name, last_name))
    
    driver.find_element_by_id('advanced-search').click()
    time.sleep(random.gauss(1, .1))
    
    driver.find_element_by_id('advs-firstName').clear()
    driver.find_element_by_id('advs-lastName').clear()
    driver.find_element_by_id('advs-school').clear()
    
    driver.find_element_by_id('advs-firstName').send_keys(first_name)
    driver.find_element_by_id('advs-lastName').send_keys(last_name)
    driver.find_element_by_id('advs-school').send_keys(school)
    driver.find_element_by_class_name('submit-advs').click()
    time.sleep(random.gauss(4.2, .5))
    
    num_results = driver.find_element_by_class_name('search-info').text.split('\n')[0]
    num_results = num_results.strip(" results")
    #num_results = int(num_results)
    if num_results == '0':
        info = (entity_id, orig_name, grad_year, business_title, employer, first_name, last_name)
        unfound_info.append(list(info))
        continue
        
    results_summary = driver.find_element_by_id('results-container').text.split('\n')
    #print('results summary: {}'.format(results_summary))
    scraped_name = results_summary[0]
    for chars in REPLACE_CHARS:
        scraped_name = scraped_name.replace(chars, '')
    #print('scraped name: {}'.format(scraped_name))
    summary_position = results_summary[1]
    
    full_name = first_name + " " + last_name
    name_match = "True" if scraped_name == full_name else "False"
    
    driver.find_element_by_css_selector('a.title.main-headline').click()
    time.sleep(random.gauss(1.1, .2))
    
    try:
        education_text = driver.find_element_by_id('background-education').text.replace('Education\n', '')
        school_match = "True" if school in education_text else "False"
        year_match = "True" if grad_year in education_text else "False"
        parsed_education_text = education_text.split('\n')
    except Exception:
        education_text = parsed_education_text = ''
        school_match = year_match = "False"
        
    try:
        work_experience = driver.find_element_by_id('background-experience-container').text.replace('Experience\n', '')
        parsed_work_experience = work_experience.split('\n')
        current_position = parsed_work_experience[0]
        current_company = parsed_work_experience[1]
        current_location = parsed_work_experience[2].rsplit(')')[1]
        employer_match = "True" if current_company == employer else "False"
    except Exception:
        parsed_work_experience = current_position = current_company = current_location = ''
        employer_match = "False"
        
    if year_match == 'False':
        info = (entity_id, orig_name, grad_year, business_title, employer, first_name, last_name, num_results, scraped_name, summary_position,
            current_position, current_company, current_location,
            parsed_education_text, parsed_work_experience, name_match, school_match, year_match, employer_match)
        unyearmatch_info.append(list(info))
        continue
        
    #if name_match == 'False' and school_match == 'False' and year_match == 'False':
    #    try_mcintyre(driver=driver, first_name, last_name, school=school2)
    
    driver.find_element_by_id('advanced-search').click()
    time.sleep(random.gauss(1.1, .1))
    
    info = (entity_id, orig_name, grad_year, business_title, employer, first_name, last_name, num_results, scraped_name, summary_position,
            current_position, current_company, current_location,
            parsed_education_text, parsed_work_experience, name_match, school_match, year_match, employer_match)
    scraped_info.append(list(info))


Processing 0: Jonathan Housley
Processing 1: Charles Verbeck
Processing 2: Miss Elaine Schweitzer
Processing 3: Kimberly Crowell
Processing 4: Christopher Lawrence
Processing 5: Joan Kearns
Processing 6: Jessica Tindel
Processing 7: Edward Testerman
Processing 8: Pete Kotarides
Processing 9: Stephen Depasquale
Processing 10: Maxwell Duncan
Processing 11: Lauren Bain
Processing 12: Robin Pitts
Processing 13: Patricia Cary
Processing 14: Stacie Zambas
Processing 15: Zenubia Madhani
Processing 16: Sara Duff McKeon
Processing 17: Brandon Furlong
Processing 18: Philip Bednar
Processing 19: Jeremy Eichler
Processing 20: Frank Williams
Processing 21: Sandra Lineberger Wolk
Processing 22: Jason Ware
Processing 23: Gregory O'Brien
Processing 24: Stephan Miller
Processing 25: Allison Lines
Processing 26: Miss Lorrie Bradley
Processing 27: Matthew Rubin
Processing 28: John Schmidt
Processing 29: Susan Scribner
Processing 30: Robert Bretz
Processing 31: Sylvia Augustyn
Processing 32: Jamie Harbeck

In [34]:
with open('results/358-476/unfound/unfoundresults358-476.csv', 'w') as output:
    scraper_writer = csv.writer(output)
    scraper_writer.writerow(['ENTITY_ID_NUMBER','PREF_MAIL_NAME','YEAR1','BUSINESS_TITLE','EMPLOYER',
                    'first_name','last_name','No. Search Results','Search Results Name',
                    'Search Results Work Summary', 'Current Position','Current Company','Current Location',
                    'Education Experience','Work Experience','Name Match','School Match','Year Match',
                    'Employer Match'])
    for row in scraped_info:
        scraper_writer.writerow(row)

In [35]:
with open('results/358-476/unfound/unfound.csv', 'w') as no_output:
    scraper_writer = csv.writer(no_output)
    scraper_writer.writerow(['ENTITY_ID_NUMBER','PREF_MAIL_NAME','YEAR1','BUSINESS_TITLE','EMPLOYER',
                    'first_name','last_name'])
    for row in unfound_info:
        scraper_writer.writerow(row)

In [36]:
with open('results/358-476/unfound/no_year_match.csv', 'w') as no_year_match:
    scraper_writer = csv.writer(no_year_match)
    scraper_writer.writerow(['ENTITY_ID_NUMBER','PREF_MAIL_NAME','YEAR1','BUSINESS_TITLE','EMPLOYER',
                    'first_name','last_name'])
    for row in unyearmatch_info:
        scraper_writer.writerow(row)